In [1]:
import redis
import re
import requests
import json
import logging
import sqlite3
import uuid
import pandas as pd

from datetime import datetime

In [2]:
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

In [3]:
api = 'localhost' # localhost
rds = '6379'
nod = '9053'
fee = 0.007 # pool fee .7%
hdr = {'api_key': 'oncejournalstrangeweather'}
minPayout = 10

red = redis.StrictRedis(host=api, port=rds, db=0, charset="utf-8", decode_responses=True)
adr = json.loads(requests.get(f'http://{api}:{nod}/mining/rewardAddress').content)['rewardAddress']
xac = json.loads(requests.get(f'http://{api}:{nod}/wallet/transactions?minInclusionHeight=0', headers=hdr).content)

DEBUG:Starting new HTTP connection (1): localhost
DEBUG:http://localhost:9053 "GET /mining/rewardAddress HTTP/1.1" 200 106
DEBUG:Starting new HTTP connection (1): localhost
DEBUG:http://localhost:9053 "GET /wallet/transactions?minInclusionHeight=0 HTTP/1.1" 200 1443119


In [4]:
db = 'payouts.db'
tbl = 'payouts'
con = sqlite3.connect(db)

Handle, "waiting" status.  Using block/miner granularity, store: 
- worker
- rig
- block (aka height)
- woker shares
- worker ergs
- block reward
- fee
- total shares for block
- total ergs for block, after fee
- timestamp for waiting status

In [5]:
# miner = worker.rig
miners = {}
blocks = {}

# search all redis keys in this format
for k in red.keys(): 
    
    # match format using regex
    m = re.search('^ergo:shares:round(?P<round>\d+)$', k)
    if m:
        round = m.group('round')
        miners[round] = {}
        blocks[round] = {}
        blocks[round]['totalShares'] = 0
        blocks[round]['fee'] = fee
        blocks[round]['rewardAmount_sat'] = 0
        blocks[round]['totalAmountAfterFee_sat'] = 0
        
        # now sum shares by miner
        shares = red.hgetall(f'ergo:shares:round{round}')
        for s in shares:
            share = int(shares[s])
            blocks[round]['totalShares'] += share
            try: 
                if s in miners[round]:
                    miners[round][s] += share # add shares
                else:
                    miners[round][s] = share # add shares
            except ValueError: 
                logging.error('not an int')
            miners[round][s] = {}
            miners[round][s]['worker'] = s.split('.')[0]
            if len(s.split('.')) == 2: 
                miners[round][s]['rig'] = s.split('.')[1]
            else:
                miners[round][s]['rig'] = ''
            miners[round][s]['shares'] = share # add shares

# fin
logging.debug(miners)
logging.debug(blocks)

 1}}, '51992': {'3Wz2aYefHaT4VmDNEm53UTr5CwkTinJStNqUbHUJuM2SEAf51SMm': {'worker': '3Wz2aYefHaT4VmDNEm53UTr5CwkTinJStNqUbHUJuM2SEAf51SMm', 'rig': '', 'shares': 2}}, '52054': {'3WvsUJxRgFeDmiFVhmkA6ysqfmRYQwsrWK57Y7BnYEsGnvRTNRbY.leif': {'worker': '3WvsUJxRgFeDmiFVhmkA6ysqfmRYQwsrWK57Y7BnYEsGnvRTNRbY', 'rig': 'leif', 'shares': 1}}, '51986': {'3Wz2aYefHaT4VmDNEm53UTr5CwkTinJStNqUbHUJuM2SEAf51SMm': {'worker': '3Wz2aYefHaT4VmDNEm53UTr5CwkTinJStNqUbHUJuM2SEAf51SMm', 'rig': '', 'shares': 5}}, '52069': {'3WvsUJxRgFeDmiFVhmkA6ysqfmRYQwsrWK57Y7BnYEsGnvRTNRbY.leif': {'worker': '3WvsUJxRgFeDmiFVhmkA6ysqfmRYQwsrWK57Y7BnYEsGnvRTNRbY', 'rig': 'leif', 'shares': 2}}, '52007': {'3Wz2aYefHaT4VmDNEm53UTr5CwkTinJStNqUbHUJuM2SEAf51SMm': {'worker': '3Wz2aYefHaT4VmDNEm53UTr5CwkTinJStNqUbHUJuM2SEAf51SMm', 'rig': '', 'shares': 1}}, '51767': {'3WvsUJxRgFeDmiFVhmkA6ysqfmRYQwsrWK57Y7BnYEsGnvRTNRbY.leif': {'worker': '3WvsUJxRgFeDmiFVhmkA6ysqfmRYQwsrWK57Y7BnYEsGnvRTNRbY', 'rig': 'leif', 'shares': 1}}, '52016': {'3W

In [6]:
for x in xac:

    # search all transactions for payments to reward address
    for o in x['outputs']:    

        # only accept with scans=9
        if 9 in x['scans']:

            # transaction details
            if o['address'] == adr:
                
                round = str(o['creationHeight'])
                if round in blocks:                    
                    blocks[round]['rewardAmount_sat'] = int(o['value']) or 0 # satoshis
                    blocks[round]['totalAmountAfterFee_sat'] = int(o['value'] - o['value']*fee) or 0

# fin
logging.debug(blocks)

DEBUG:{'52034': {'totalShares': 2, 'fee': 0.007, 'rewardAmount_sat': 67500000000, 'totalAmountAfterFee_sat': 67027500000}, '51985': {'totalShares': 1, 'fee': 0.007, 'rewardAmount_sat': 67500000000, 'totalAmountAfterFee_sat': 67027500000}, '51763': {'totalShares': 2, 'fee': 0.007, 'rewardAmount_sat': 67500000000, 'totalAmountAfterFee_sat': 67027500000}, '52057': {'totalShares': 1, 'fee': 0.007, 'rewardAmount_sat': 67500000000, 'totalAmountAfterFee_sat': 67027500000}, '51988': {'totalShares': 2, 'fee': 0.007, 'rewardAmount_sat': 67500000000, 'totalAmountAfterFee_sat': 67027500000}, '52027': {'totalShares': 1, 'fee': 0.007, 'rewardAmount_sat': 67500000000, 'totalAmountAfterFee_sat': 67027500000}, '52024': {'totalShares': 1, 'fee': 0.007, 'rewardAmount_sat': 67500000000, 'totalAmountAfterFee_sat': 67027500000}, '52060': {'totalShares': 1, 'fee': 0.007, 'rewardAmount_sat': 67500000000, 'totalAmountAfterFee_sat': 67027500000}, '51989': {'totalShares': 1, 'fee': 0.007, 'rewardAmount_sat': 675

All stats for waiting found; store in db

In [ ]:
db = 'payouts.db'
with sqlite3.connect(db) as sql:
    sql.execute("drop table payouts")

with sqlite3.connect(db) as sql:
    sql.execute("""
        create table if not exists payouts (
            id integer not null primary key autoincrement,
            block text,
            miner text,
            worker text,
            rig text,
            status text,
            workerShares integer,
            workerShares_erg integer,
            blockReward_sat integer,
            poolFee_pct real,
            totalBlockShares integer,
            totalAmountAfterFee_sat integer,
            pendingBatchId text,
            payoutBatchAmount_erg real,
            paidTransactionId text,
            _timestampWaiting datetime,
            _timestampPending datetime,
            _timestampPaid datetime
        )
    """)


In [ ]:
with sqlite3.connect(db) as sql:
    sql.execute("""insert into payouts (block, miner, worker, rig, status, workerShares, workerShares_erg, blockReward_sat, poolFee_pct, totalBlockShares, totalAmountAfterFee_sat, pendingBatchId, payoutBatchAmount_erg, paidTransactionId, _timestampWaiting, _timestampPending, _timestampPaid)
        values (
            '123abc456def'
            , 'minername.rig'
            , 'minername'
            , 'rig'
            , '_testing'
            , 33
            , 45.5
            , 100000000000
            , 0.01
            , 66
            , 99000000000
            , '1234-5678-abcd-0000'
            , 91.1
            , 'xyzpdq'
            , strftime('%Y-%m-%dT%H:%M:%f', 'now')
            , datetime('now')
            , julianday('now')
        )
        """)

# with sqlite3.connect(db) as sql:
#     for r in sql.execute('select * from payouts'):
#         logging.debug(r)
    
df = pd.read_sql_query('select * from payouts', con=con)
logging.debug(df)

# reset
with sqlite3.connect(db) as sql: sql.execute("drop table payouts")
for round in [51762, 51763, 51765, 51766, 51767, 51768]: red.rename(f'ergo:shares:payout{round}', f'ergo:shares:round{round}')

In [15]:
rows = []
rounds = {}
for block in blocks:
    for miner in miners[block]:
        if blocks[block]['rewardAmount_sat'] > 0:
            totalAmountAfterFee_sat = int(blocks[block]['rewardAmount_sat']) - (int(blocks[block]['rewardAmount_sat'])*fee)
            workerShares_erg = (int(miners[block][miner]['shares'])/int(blocks[block]['totalShares'])) * totalAmountAfterFee_sat / 1000000000
            rows.append([block, miner, miners[block][miner]['worker'], miners[block][miner]['rig'], 'waiting', miners[block][miner]['shares'], workerShares_erg, blocks[block]['rewardAmount_sat'], fee, blocks[block]['totalShares'], totalAmountAfterFee_sat, '', 0.0, '', datetime.now().isoformat(), '', ''])    
            rounds[block] = 0
df = pd.DataFrame(rows, columns=['block', ' miner', ' worker', ' rig', ' status', ' workerShares', ' workerShares_erg', ' blockReward_sat', ' poolFee_pct', ' totalBlockShares', 'totalAmountAfterFee_sat', ' pendingBatchId', ' payoutBatchAmount_erg', ' paidTransactionId', ' _timestampWaiting', ' _timestampPending', ' _timestampPaid'])
print(rounds)

{'52034': 0, '51985': 0, '51763': 0, '52057': 0, '51988': 0, '52027': 0, '52024': 0, '52060': 0, '51989': 0, '52040': 0, '52065': 0, '52061': 0, '52005': 0, '52037': 0, '51983': 0, '52052': 0, '51998': 0, '52079': 0, '52048': 0, '52047': 0, '52074': 0, '52083': 0, '52004': 0, '51987': 0, '52044': 0, '52000': 0, '52025': 0, '52031': 0, '52020': 0, '51984': 0, '52026': 0, '52001': 0, '52086': 0, '51762': 0, '52062': 0, '52042': 0, '52059': 0, '52015': 0, '52035': 0, '52078': 0, '51990': 0, '52006': 0, '52082': 0, '52028': 0, '52049': 0, '52063': 0, '52030': 0, '52032': 0, '52012': 0, '52066': 0, '52003': 0, '52013': 0, '52014': 0, '52021': 0, '52051': 0, '51992': 0, '52054': 0, '51986': 0, '52069': 0, '52007': 0, '51767': 0, '52016': 0, '52011': 0, '52036': 0, '52010': 0, '51993': 0, '52008': 0, '52064': 0, '52077': 0, '52045': 0, '52053': 0, '51997': 0, '52023': 0, '51768': 0, '52081': 0, '52038': 0, '52080': 0, '52056': 0, '51991': 0, '52018': 0, '52050': 0, '52073': 0, '52076': 0, '52

In [13]:
rows = []
for block in blocks:
    for miner in miners[block]:
        if blocks[block]['rewardAmount_sat'] > 0:
            totalAmountAfterFee_sat = int(blocks[block]['rewardAmount_sat']) - (int(blocks[block]['rewardAmount_sat'])*fee)
            workerShares_erg = (int(miners[block][miner]['shares'])/int(blocks[block]['totalShares'])) * totalAmountAfterFee_sat / 1000000000
            rows.append([block, miner, miners[block][miner]['worker'], miners[block][miner]['rig'], 'waiting', miners[block][miner]['shares'], workerShares_erg, blocks[block]['rewardAmount_sat'], fee, blocks[block]['totalShares'], totalAmountAfterFee_sat, '', 0.0, '', datetime.now().isoformat(), '', ''])

try :
    # add new shares to waiting status
    logging.info('saving new shares to database...')
    df = pd.DataFrame(rows, columns=['block', 'miner', 'worker', 'rig', 'status', 'workerShares', 'workerShares_erg', 'blockReward_sat', 'poolFee_pct', 'totalBlockShares', 'totalAmountAfterFee_sat', 'pendingBatchId', 'payoutBatchAmount_erg', 'paidTransactionId', '_timestampWaiting', '_timestampPending', '_timestampPaid'])
    if len(df) > 0:
        df.to_sql('payouts', if_exists='append', con=con, index=False)
        
        # update rounds so that are not counted again
        for round in rounds:
            logging.info(f'renaming redis key, ergo:shares:round{round}...')
            red.rename(f'ergo:shares:round{round}', f'ergo:shares:payout{round}')

except Exception as e:
    logging.error(e)

# df.to_sql(tbl, con)

INFO:saving new shares to database...
INFO:renaming redis key, ergo:shares:round52034...
INFO:renaming redis key, ergo:shares:round51985...
INFO:renaming redis key, ergo:shares:round51763...
INFO:renaming redis key, ergo:shares:round52057...
INFO:renaming redis key, ergo:shares:round51988...
INFO:renaming redis key, ergo:shares:round52027...
INFO:renaming redis key, ergo:shares:round52024...
INFO:renaming redis key, ergo:shares:round52060...
INFO:renaming redis key, ergo:shares:round51989...
INFO:renaming redis key, ergo:shares:round52040...
INFO:renaming redis key, ergo:shares:round52065...
INFO:renaming redis key, ergo:shares:round52061...
INFO:renaming redis key, ergo:shares:round52005...
INFO:renaming redis key, ergo:shares:round52037...
INFO:renaming redis key, ergo:shares:round51983...
INFO:renaming redis key, ergo:shares:round52052...
INFO:renaming redis key, ergo:shares:round51998...
INFO:renaming redis key, ergo:shares:round52079...
INFO:renaming redis key, ergo:shares:round52

In [22]:
try:
    df = pd.read_sql_query("select * from payouts where status = 'waiting'", con=con)
    dfTotals = df.groupby(['worker'])['workerShares_erg'].sum().reset_index()
    dfPending = dfTotals[dfTotals['workerShares_erg'] >= minPayout]

    for r in dfPending.itertuples():
        logging.info(f'make payment to worker, {r.worker}...')

        batch = str(uuid.uuid4())
        logging.debug(f'log payment info for {r.worker}, batch: {batch}, amount: {r.workerShares_erg}ergs')
        bdy = [{'address': r.worker, 'value': int(r.workerShares_erg*1000000000), 'assets': []}]
        res = requests.post(f'http://{api}:{nod}/wallet/payment/send', headers=hdr, json=bdy)
        with sqlite3.connect(db) as sql:
            sql.execute(f"""
                update {tbl} 
                set pendingBatchId = '{batch}'
                    , payoutBatchAmount_erg = {r.workerShares_erg}
                    , _timestampPending = '{datetime.now().isoformat()}'
                    , paidTransactionId = '{json.loads(res.content)}'
                    , status = 'pending'
                where worker = '{r.worker}'
                    and status = 'waiting'
                """)

except Exception as e:
    logging.error(f'handlePayouts::{e}')


INFO:make payment to worker, 3WvsUJxRgFeDmiFVhmkA6ysqfmRYQwsrWK57Y7BnYEsGnvRTNRbY...
DEBUG:log payment info for 3WvsUJxRgFeDmiFVhmkA6ysqfmRYQwsrWK57Y7BnYEsGnvRTNRbY, batch: bd54eaa0-5959-4f36-b008-43d97e2544c3, amount: 3641.8275000000026ergs
DEBUG:Starting new HTTP connection (1): localhost
DEBUG:http://localhost:9053 "POST /wallet/payment/send HTTP/1.1" 200 66
INFO:make payment to worker, 3Wz2aYefHaT4VmDNEm53UTr5CwkTinJStNqUbHUJuM2SEAf51SMm...
DEBUG:log payment info for 3Wz2aYefHaT4VmDNEm53UTr5CwkTinJStNqUbHUJuM2SEAf51SMm, batch: 8b53a366-d21b-4109-87bf-20c524c6e514, amount: 3798.2250000000026ergs
DEBUG:Starting new HTTP connection (1): localhost
DEBUG:http://localhost:9053 "POST /wallet/payment/send HTTP/1.1" 200 66
